In [60]:
#1
import numpy as np

# تعریف پارامترها
L = 0.61  # طول سیستم
s_in = 0.01
s_out = 2 * L

pi = np.pi
num_steps = 10000

# محاسبات اولیه
k = (2 * pi) / L
K = k**2 / 8

# محاسبه A_H و A_L
A_L = 0.00750
A_H = 0.0080
N = 1

# مقادیر g و g_p و g_m
g = K * ((L / pi) ** 2)
g_p = np.sqrt(1 + g)
g_m = np.sqrt(1 - g)

# تابع مثلثاتی مشترک
def trig_functions(s, g_p, g_m, L):
    bar_s = (np.pi * s) / L
    Cp = np.cos(g_p * bar_s)
    Cm = np.cos(g_m * bar_s)
    Sp = np.sin(g_p * bar_s)
    Sm = np.sin(g_m * bar_s)
    return Cp, Cm, Sp, Sm

# محاسبه ماتریس M_core
def compute_M_core(L, g, g_p, g_m, s):
    Cp, Cm, Sp, Sm = trig_functions(s, g_p, g_m, L)
    M_core = np.array([
        [Cp, (L / (g * np.pi)) * ((g_p * Sp) - (Sm / g_m)), Sm / g_m, (L / (g * np.pi)) * (Cm - Cp)],
        [(-np.pi * g * Sp) / (L * g_p), Cp, 0, (Sp / g_p)],
        [(-Sp / g_p), (L / (g * np.pi)) * (Cp - Cm), Cm, (L / (g * np.pi)) * ((Sp / g_p) - (g_m * Sm))],
        [0, -(Sm / g_m), (np.pi * g * Sm) / (L * g_m), Cm]
    ])
    return M_core

# محاسبه ماتریس‌های چرخش R_positive و R_negative
def compute_R_positive(k, s):
    return np.array([
        [np.cos(k * s / 2), 0, -np.sin(k * s / 2), 0],
        [0, np.cos(k * s / 2), 0, -np.sin(k * s / 2)],
        [np.sin(k * s / 2), 0, np.cos(k * s / 2), 0],
        [0, np.sin(k * s / 2), 0, np.cos(k * s / 2)]
    ])

def compute_R_negative(k, s):
    return np.array([
        [np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N), 0],
        [0, np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N)],
        [-np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N), 0],
        [0, -np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N)]
    ])

# محاسبه ماتریس انتقال M_HQ
def helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m):
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    M_core = compute_M_core(L, g, g_p, g_m, s_out - s_in)
    M_HQ = R_out @ M_core @ R_in 
    return M_HQ

# تابع محاسبه‌ی ماتریس سیگما داخلی
def sigma_inner_matrix(k, K, A_H, A_L):
    sigma = np.zeros((4, 4))
    sigma[0, 0] = ((4 * k**2 + 4 * K) /(k**2)) * A_H**2 + (k**2 / K**2) * A_L**2
    sigma[1, 1] = 16 * (K**2 / k**2) * A_H**2
    sigma[2, 2] = 4 * A_H**2 + ((k**2 - 4 * K) / K**2) * A_L**2
    sigma[3, 3] = 4 * A_L**2
    sigma[0, 3] = sigma[3, 0] = 2 * (k / K) * A_L**2
    sigma[1, 2] = sigma[2, 1] = 8 * (K / k) * A_H**2
    return sigma

# اعمال ماتریس سیگما و محاسبه نهایی با ضرب ماتریس‌های چرخش
def sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out):
    sigma_inner = sigma_inner_matrix(k, K, A_H, A_L)
    
    # محاسبه ماتریس‌های چرخش
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    
    # اعمال چرخش‌ها به سیگما
    sigma_rotated = R_out @ sigma_inner @ R_in
    return sigma_rotated
    
    # محاسبه سیگمای نهایی با ضرب در M_HQ
    sigma_final = M_HQ @ sigma_rotated @ M_HQ.T
    return sigma_final 

# محاسبه انتشار
def compute_emittance(cov_matrix):
    return np.sqrt(np.linalg.det(cov_matrix))

# محاسبه پارامترهای توییس
def compute_twist_parameters(cov_matrix, emittance):
    beta = cov_matrix[0, 0] / emittance
    alpha = -cov_matrix[0, 1] / emittance
    gamma = cov_matrix[1, 1] / emittance
    return beta, alpha, gamma

# محاسبه M_HQ
M_HQ = helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m)

# محاسبه ماتریس سیگمای اولیه
sigma_initial = sigma_inner_matrix(k, K, A_H, A_L)

sigma_rotated = sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های اولیه
cov_x_initial = sigma_rotated[:2, :2]
cov_y_initial = sigma_rotated[2:, 2:]


# محاسبه امیتنس‌های اولیه
emittance_x_initial = compute_emittance(cov_x_initial)
emittance_y_initial = compute_emittance(cov_y_initial)

# محاسبه پارامترهای توییس برای امیتنس‌های اولیه
beta_x_initial, alpha_x_initial, gamma_x_initial = compute_twist_parameters(cov_x_initial, emittance_x_initial)
beta_y_initial, alpha_y_initial, gamma_y_initial = compute_twist_parameters(cov_y_initial, emittance_y_initial)

# نمایش امیتنس‌های اولیه
print("\nInitial Emittance for x-plane:")
print(f"Emittance: {emittance_x_initial:.4f}")
print(f"Beta: {beta_x_initial:.4f}, Alpha: {alpha_x_initial:.4f}, Gamma: {gamma_x_initial:.4f}")

print("\nInitial Emittance for y-plane:")
print(f"Emittance: {emittance_y_initial:.4f}")
print(f"Beta: {beta_y_initial:.4f}, Alpha: {alpha_y_initial:.4f}, Gamma: {gamma_y_initial:.4f}")

# محاسبه ماتریس سیگما نهایی
sigma_final = sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های نهایی
cov_x_final = sigma_final[:2, :2]
cov_y_final = sigma_final[2:, 2:]

# محاسبه امیتنس‌های نهایی
emittance_x_final = compute_emittance(cov_x_final)
emittance_y_final = compute_emittance(cov_y_final)

# محاسبه پارامترهای توییس برای امیتنس‌های نهایی
beta_x_final, alpha_x_final, gamma_x_final = compute_twist_parameters(cov_x_final, emittance_x_final)
beta_y_final, alpha_y_final, gamma_y_final = compute_twist_parameters(cov_y_final, emittance_y_final)

# نمایش نتایج نهایی برای x
print("\nFinal Twiss Parameters for x-plane (Transfer Matrix):")
print(f"Emittance: {emittance_x_final:.4f}")
print(f"Beta: {beta_x_final:.4f}, Alpha: {alpha_x_final:.4f}, Gamma: {gamma_x_final:.4f}")

# نمایش نتایج نهایی برای y
print("\nFinal Twiss Parameters for y-plane (Transfer Matrix):")
print(f"Emittance: {emittance_y_final:.4f}")
print(f"Beta: {beta_y_final:.4f}, Alpha: {alpha_y_final:.4f}, Gamma: {gamma_y_final:.4f}")



Initial Emittance for x-plane:
Emittance: 0.0007
Beta: 0.4355, Alpha: 0.0061, Gamma: 2.2966

Initial Emittance for y-plane:
Emittance: 0.0002
Beta: 1.1028, Alpha: -0.1373, Gamma: 0.9090

Final Twiss Parameters for x-plane (Transfer Matrix):
Emittance: 0.0007
Beta: 0.4355, Alpha: 0.0061, Gamma: 2.2966

Final Twiss Parameters for y-plane (Transfer Matrix):
Emittance: 0.0002
Beta: 1.1028, Alpha: -0.1373, Gamma: 0.9090


In [8]:
#2 
import numpy as np

# تابع محاسبه‌ی امیتنس با استفاده از روش کواریانس
def compute_emittance(cov_matrix):
    return np.sqrt(np.linalg.det(cov_matrix))

# تابع محاسبه‌ی پارامترهای توییس
def compute_twist_parameters(cov_matrix, emittance):
    beta = cov_matrix[0, 0] / emittance
    alpha = -cov_matrix[0, 1] / emittance
    gamma = cov_matrix[1, 1] / emittance
    return beta, alpha, gamma

# تعریف تابع محاسبه ماتریس سیگما داخلی
def sigma_inner_matrix(k, K, A_H, A_L):
    sigma = np.zeros((4, 4))
    sigma[0, 0] = ((4 * k**2 + 4 * K) /(k**2)) * A_H**2 + (k**2 / K**2) * A_L**2
    sigma[1, 1] = 16 * (K**2 / k**2) * A_H**2
    sigma[2, 2] = 4 * A_H**2 + ((k**2 - 4 * K) / K**2) * A_L**2
    sigma[3, 3] = 4 * A_L**2
    sigma[0, 3] = sigma[3, 0] = 2 * (k / K) * A_L**2
    sigma[1, 2] = sigma[2, 1] = 8 * (K / k) * A_H**2
    return sigma

# محاسبه ماتریس‌های چرخش R_positive و R_negative
def compute_R_positive(k, s):
    return np.array([
        [np.cos(k * s / 2), 0, -np.sin(k * s / 2), 0],
        [0, np.cos(k * s / 2), 0, -np.sin(k * s / 2)],
        [np.sin(k * s / 2), 0, np.cos(k * s / 2), 0],
        [0, np.sin(k * s / 2), 0, np.cos(k * s / 2)]
    ])

def compute_R_negative(k, s):
    return np.array([
        [np.cos(k * s / 2), 0, np.sin(k * s / 2), 0],
        [0, np.cos(k * s / 2), 0, np.sin(k * s / 2)],
        [-np.sin(k * s / 2), 0, np.cos(k * s / 2), 0],
        [0, -np.sin(k * s / 2), 0, np.cos(k * s / 2)]
    ])

# محاسبه ماتریس انتقال (M_HQ) با فرمولی که در بالا بود
def helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m):
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    M_core = compute_M_core(L, g, g_p, g_m, s_out - s_in)
    M_HQ = R_out @ M_core @ R_in
    return M_HQ

# تابع محاسبه M_core
def compute_M_core(L, g, g_p, g_m, s):
    bar_s = (np.pi * s) / L
    Cp = np.cos(g_p * bar_s)
    Cm = np.cos(g_m * bar_s)
    Sp = np.sin(g_p * bar_s)
    Sm = np.sin(g_m * bar_s)
    M_core = np.array([
        [Cp, (L / (g * np.pi)) * ((g_p * Sp) - (Sm / g_m)), Sm / g_m, (L / (g * np.pi)) * (Cm - Cp)],
        [(-np.pi * g * Sp) / (L * g_p), Cp, 0, (Sp / g_p)],
        [(-Sp / g_p), (L / (g * np.pi)) * (Cp - Cm), Cm, (L / (g * np.pi)) * ((Sp / g_p) - (g_m * Sm))],
        [0, -(Sm / g_m), (np.pi * g * Sm) / (L * g_m), Cm]
    ])
    return M_core

# تعریف پارامترها برای سیستم شما
L = 0.61  # طول سیستم
s_in = 0.01
s_out = 2 * L
pi = np.pi
k = (2 * pi) / L
K = k**2 / 8

# مقادیر g و g_p و g_m
g = K * ((L / pi) ** 2)
g_p = np.sqrt(1 + g)
g_m = np.sqrt(1 - g)

# مقادیر A_L و A_H (مقادیر امیتنس اولیه)
A_L = 0.00750
A_H = 0.0080

# محاسبه ماتریس سیگمای اولیه
sigma_initial = sigma_inner_matrix(k, K, A_H, A_L)

# محاسبه امیتنس‌های اولیه
cov_x_initial = sigma_initial[:2, :2]
cov_y_initial = sigma_initial[2:, 2:]

emittance_x_initial = compute_emittance(cov_x_initial)
emittance_y_initial = compute_emittance(cov_y_initial)

# محاسبه پارامترهای توییس اولیه
beta_x_initial, alpha_x_initial, gamma_x_initial = compute_twist_parameters(cov_x_initial, emittance_x_initial)
beta_y_initial, alpha_y_initial, gamma_y_initial = compute_twist_parameters(cov_y_initial, emittance_y_initial)

print("\nInitial Emittance for x-plane:")
print(f"Emittance: {emittance_x_initial:.4f}")
print(f"Initial Beta: {beta_x_initial:.4f}, Alpha: {alpha_x_initial:.4f}, Gamma: {gamma_x_initial:.4f}")

print("\nInitial Emittance for y-plane:")
print(f"Emittance: {emittance_y_initial:.4f}")
print(f"Initial Beta: {beta_y_initial:.4f}, Alpha: {alpha_y_initial:.4f}, Gamma: {gamma_y_initial:.4f}")

# حالا می‌خواهیم ترکینگ کنیم: یعنی با استفاده از مقادیر اولیه‌ی x و y و ماتریس انتقال، مقادیر جدید را حساب کنیم

num_particles = 1000  # تعداد ذرات
x_init_values = np.random.normal(0.05, 0.034, num_particles)
x_prime_init_values = np.random.normal(0.01, 0.0099, num_particles)
y_init_values = np.random.normal(0.010, 0.014, num_particles)
y_prime_init_values = np.random.normal(0.01, 0.0070, num_particles)

# تعریف تابع ترکینگ ذرات (با استفاده از M_HQ روی داده‌های اولیه)
def track_particles(M_HQ, x_init, x_prime_init, y_init, y_prime_init):
    x_new = M_HQ[0, 0] * x_init + M_HQ[0, 1] * x_prime_init
    x_prime_new = M_HQ[1, 0] * x_init + M_HQ[1, 1] * x_prime_init
    y_new = M_HQ[2, 2] * y_init + M_HQ[2, 3] * y_prime_init
    y_prime_new = M_HQ[3, 2] * y_init + M_HQ[3, 3] * y_prime_init
    return x_new, x_prime_new, y_new, y_prime_new

# محاسبه ماتریس M_HQ با استفاده از مقادیر اولیه
M_HQ = helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m)

# اعمال ماتریس انتقال به داده‌های x و y برای ترکینگ
x_new_values, x_prime_new_values, y_new_values, y_prime_new_values = track_particles(
    M_HQ, x_init_values, x_prime_init_values, y_init_values, y_prime_init_values
)

# اکنون باید امیتنس نهایی را محاسبه کنیم
def compute_emittance_from_data(positions, angles):
    data = np.vstack((positions, angles))
    covariance_matrix = np.cov(data)
    sigma_xx = covariance_matrix[0, 0]
    sigma_xp = covariance_matrix[0, 1]
    sigma_pp = covariance_matrix[1, 1]
    emittance = np.sqrt(sigma_xx * sigma_pp - sigma_xp**2)
    return emittance, covariance_matrix

# محاسبه امیتنس‌های نهایی برای x و y
emittance_x_final, cov_x_final = compute_emittance_from_data(x_new_values, x_prime_new_values)
emittance_y_final, cov_y_final = compute_emittance_from_data(y_new_values, y_prime_new_values)

# محاسبه پارامترهای توییس نهایی
beta_x_final, alpha_x_final, gamma_x_final = compute_twist_parameters(cov_x_final, emittance_x_final)
beta_y_final, alpha_y_final, gamma_y_final = compute_twist_parameters(cov_y_final, emittance_y_final)

# نمایش امیتنس‌ها و پارامترهای توییس نهایی
print(f"\nFinal Emittance for x-plane (after tracking): {emittance_x_final:.6f}")
print(f"Final Beta: {beta_x_final:.4f}, Alpha: {alpha_x_final:.4f}, Gamma: {gamma_x_final:.4f}")

print(f"\nFinal Emittance for y-plane (after tracking): {emittance_y_final:.6f}")
print(f"Final Beta: {beta_y_final:.4f}, Alpha: {alpha_y_final:.4f}, Gamma: {gamma_y_final:.4f}")



Initial Emittance for x-plane:
Emittance: 0.0007
Initial Beta: 0.4355, Alpha: -0.0000, Gamma: 2.2963

Initial Emittance for y-plane:
Emittance: 0.0002
Initial Beta: 1.1014, Alpha: -0.0000, Gamma: 0.9079

Final Emittance for x-plane (after tracking): 0.000668
Final Beta: 0.2736, Alpha: 0.9163, Gamma: 6.7242

Final Emittance for y-plane (after tracking): 0.000209
Final Beta: 0.1832, Alpha: -1.0238, Gamma: 11.1792


In [1]:
#کد با اضافه کردن space charge
import numpy as np

# تعریف پارامترها
L = 0.61  # طول سیستم
s_in = 0.01
s_out = 2 * L
pi = np.pi
num_steps = 10000

# پارامترهای شارژ فضایی
I = 1e-6  # جریان پرتو (آمپر)
gamma = 100  # فاکتور نسبیتی
epsilon_0 = 8.854187817e-12  # ثابت دی‌الکتریک خلاء
a = 0.01  # شعاع پرتو در محور x
b = 0.01  # شعاع پرتو در محور y

# ضریب برای کاهش یا افزایش اثر شارژ فضایی
space_charge_factor = 0.0001  # ابتدا اثر کوچک، سپس می‌توان افزایش داد

# محاسبات اولیه
k = (2 * pi) / L
K = k**2 / 8

# محاسبه A_H و A_L
A_L = 0.00750
A_H = 0.0080
N = 1

# محاسبه میدان‌های الکتریکی ناشی از شارژ فضایی
E_x_space_charge = space_charge_factor * (4 * I) / (4 * pi * epsilon_0 * gamma**2 * a * (a + b))
E_y_space_charge = space_charge_factor * (4 * I) / (4 * pi * epsilon_0 * gamma**2 * b * (a + b))

# تصحیح Kx و Ky با اثر شارژ فضایی
delta_Kx = E_x_space_charge / (k * (a + b))
delta_Ky = E_y_space_charge / (k * (a + b))

# مقادیر g و g_p و g_m
g = K * ((L / pi) ** 2)
g_p = np.sqrt(1 + g)
g_m = np.sqrt(1 - g)

# تابع مثلثاتی مشترک
def trig_functions(s, g_p, g_m, L):
    bar_s = (np.pi * s) / L
    Cp = np.cos(g_p * bar_s)
    Cm = np.cos(g_m * bar_s)
    Sp = np.sin(g_p * bar_s)
    Sm = np.sin(g_m * bar_s)
    return Cp, Cm, Sp, Sm

# محاسبه ماتریس M_core
def compute_M_core(L, g, g_p, g_m, s, delta_Kx, delta_Ky):
    Cp, Cm, Sp, Sm = trig_functions(s, g_p, g_m, L)
    M_core = np.array([
        [Cp, (L / (g * np.pi)) * ((g_p * Sp) - (Sm / g_m)), Sm / g_m, (L / (g * np.pi)) * (Cm - Cp)],
        [(-np.pi * g * Sp) / (L * g_p), Cp - delta_Kx, 0, (Sp / g_p)],
        [(-Sp / g_p), (L / (g * np.pi)) * (Cp - Cm), Cm - delta_Ky, (L / (g * np.pi)) * ((Sp / g_p) - (g_m * Sm))],
        [0, -(Sm / g_m), (np.pi * g * Sm) / (L * g_m), Cm]
    ])
    return M_core

# محاسبه ماتریس‌های چرخش R_positive و R_negative
def compute_R_positive(k, s):
    return np.array([
        [np.cos(k * s / 2), 0, -np.sin(k * s / 2), 0],
        [0, np.cos(k * s / 2), 0, -np.sin(k * s / 2)],
        [np.sin(k * s / 2), 0, np.cos(k * s / 2), 0],
        [0, np.sin(k * s / 2), 0, np.cos(k * s / 2)]
    ])

def compute_R_negative(k, s):
    return np.array([
        [np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N), 0],
        [0, np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N)],
        [-np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N), 0],
        [0, -np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N)]
    ])

# محاسبه ماتریس انتقال M_HQ
def helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m, delta_Kx, delta_Ky):
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    M_core = compute_M_core(L, g, g_p, g_m, s_out - s_in, delta_Kx, delta_Ky)
    M_HQ = R_out @ M_core @ R_in 
    return M_HQ

# تابع محاسبه‌ی ماتریس سیگما داخلی
def sigma_inner_matrix(k, K, A_H, A_L):
    sigma = np.zeros((4, 4))
    sigma[0, 0] = ((4 * k**2 + 4 * K) /(k**2)) * A_H**2 + (k**2 / K**2) * A_L**2
    sigma[1, 1] = 16 * (K**2 / k**2) * A_H**2
    sigma[2, 2] = 4 * A_H**2 + ((k**2 - 4 * K) / K**2) * A_L**2
    sigma[3, 3] = 4 * A_L**2
    sigma[0, 3] = sigma[3, 0] = 2 * (k / K) * A_L**2
    sigma[1, 2] = sigma[2, 1] = 8 * (K / k) * A_H**2
    return sigma

# اعمال ماتریس سیگما و محاسبه نهایی با ضرب ماتریس‌های چرخش
def sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out):
    sigma_inner = sigma_inner_matrix(k, K, A_H, A_L)
    
    # محاسبه ماتریس‌های چرخش
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    
    # اعمال چرخش‌ها به سیگما
    sigma_rotated = R_out @ sigma_inner @ R_in
    
    # محاسبه سیگمای نهایی با ضرب در M_HQ
    sigma_final = M_HQ @ sigma_rotated @ M_HQ.T
    return sigma_final

# محاسبه انتشار
def compute_emittance(cov_matrix):
    return np.sqrt(np.linalg.det(cov_matrix))

# محاسبه پارامترهای توییس
def compute_twist_parameters(cov_matrix, emittance):
    beta = cov_matrix[0, 0] / emittance
    alpha = -cov_matrix[0, 1] / emittance
    gamma = cov_matrix[1, 1] / emittance
    return beta, alpha, gamma

# محاسبه M_HQ
M_HQ = helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m, delta_Kx, delta_Ky)

# محاسبه ماتریس سیگمای اولیه
sigma_initial = sigma_inner_matrix(k, K, A_H, A_L)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های اولیه
cov_x_initial = sigma_initial[:2, :2]
cov_y_initial = sigma_initial[2:, 2:]

# محاسبه امیتنس‌های اولیه
emittance_x_initial = compute_emittance(cov_x_initial)
emittance_y_initial = compute_emittance(cov_y_initial)

# محاسبه پارامترهای توییس برای امیتنس‌های اولیه
beta_x_initial, alpha_x_initial, gamma_x_initial = compute_twist_parameters(cov_x_initial, emittance_x_initial)
beta_y_initial, alpha_y_initial, gamma_y_initial = compute_twist_parameters(cov_y_initial, emittance_y_initial)

# نمایش امیتنس‌های اولیه
print("\nInitial Emittance for x-plane:")
print(f"Emittance: {emittance_x_initial:.4f}")
print(f"Beta: {beta_x_initial:.4f}, Alpha: {alpha_x_initial:.4f}, Gamma: {gamma_x_initial:.4f}")

print("\nInitial Emittance for y-plane:")
print(f"Emittance: {emittance_y_initial:.4f}")
print(f"Beta: {beta_y_initial:.4f}, Alpha: {alpha_y_initial:.4f}, Gamma: {gamma_y_initial:.4f}")

# محاسبه سیگمای نهایی
sigma_final = sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های نهایی
cov_x_final = sigma_final[:2, :2]
cov_y_final = sigma_final[2:, 2:]

# محاسبه امیتنس‌های نهایی
emittance_x_final = compute_emittance(cov_x_final)
emittance_y_final = compute_emittance(cov_y_final)

# محاسبه پارامترهای توییس برای امیتنس‌های نهایی
beta_x_final, alpha_x_final, gamma_x_final = compute_twist_parameters(cov_x_final, emittance_x_final)
beta_y_final, alpha_y_final, gamma_y_final = compute_twist_parameters(cov_y_final, emittance_y_final)

# نمایش نتایج نهایی برای x
print("\nFinal Twiss Parameters for x-plane (Transfer Matrix):")
print(f"Emittance: {emittance_x_final:.4f}")
print(f"Beta: {beta_x_final:.4f}, Alpha: {alpha_x_final:.4f}, Gamma: {gamma_x_final:.4f}")

# نمایش نتایج نهایی برای y
print("\nFinal Twiss Parameters for y-plane (Transfer Matrix):")
print(f"Emittance: {emittance_y_final:.4f}")
print(f"Beta: {beta_y_final:.4f}, Alpha: {alpha_y_final:.4f}, Gamma: {gamma_y_final:.4f}")



Initial Emittance for x-plane:
Emittance: 0.0007
Beta: 0.4355, Alpha: -0.0000, Gamma: 2.2963

Initial Emittance for y-plane:
Emittance: 0.0002
Beta: 1.1014, Alpha: -0.0000, Gamma: 0.9079

Final Twiss Parameters for x-plane (Transfer Matrix):
Emittance: 0.0022
Beta: 0.1963, Alpha: 3.2560, Gamma: 57.5462

Final Twiss Parameters for y-plane (Transfer Matrix):
Emittance: 0.0020
Beta: 9.9238, Alpha: -0.2466, Gamma: 0.1056


In [17]:
#کد با اضافه کردن space charge
import numpy as np

# تعریف پارامترها
L = 0.61  # طول سیستم
s_in = 0.01
s_out = 2 * L
pi = np.pi
num_steps = 10000

# پارامترهای شارژ فضایی
I = 1e-6  # جریان پرتو (آمپر)
gamma = 100  # فاکتور نسبیتی
epsilon_0 = 8.854187817e-12  # ثابت دی‌الکتریک خلاء
a = 0.01  # شعاع پرتو در محور x
b = 0.01  # شعاع پرتو در محور y

# ضریب برای کاهش یا افزایش اثر شارژ فضایی
space_charge_factor = 0.0001  # ابتدا اثر کوچک، سپس می‌توان افزایش داد

# محاسبات اولیه
k = (2 * pi) / L
K = k**2 / 8

# محاسبه A_H و A_L
A_L = 0.00750
A_H = 0.0080
N = 1

# محاسبه میدان‌های الکتریکی ناشی از شارژ فضایی
E_x_space_charge = space_charge_factor * (4 * I) / (4 * pi * epsilon_0 * gamma**2 * a * (a + b))
E_y_space_charge = space_charge_factor * (4 * I) / (4 * pi * epsilon_0 * gamma**2 * b * (a + b))

# تصحیح Kx و Ky با اثر شارژ فضایی
delta_Kx = E_x_space_charge / (k * (a + b))
delta_Ky = E_y_space_charge / (k * (a + b))

# مقادیر g و g_p و g_m
g = K * ((L / pi) ** 2)
g_p = np.sqrt(1 + g)
g_m = np.sqrt(1 - g)

# تابع مثلثاتی مشترک
def trig_functions(s, g_p, g_m, L):
    bar_s = (np.pi * s) / L
    Cp = np.cos(g_p * bar_s)
    Cm = np.cos(g_m * bar_s)
    Sp = np.sin(g_p * bar_s)
    Sm = np.sin(g_m * bar_s)
    return Cp, Cm, Sp, Sm

# محاسبه ماتریس M_core
def compute_M_core(L, g, g_p, g_m, s, delta_Kx, delta_Ky):
    Cp, Cm, Sp, Sm = trig_functions(s, g_p, g_m, L)
    M_core = np.array([
        [Cp, (L / (g * np.pi)) * ((g_p * Sp) - (Sm / g_m)), Sm / g_m, (L / (g * np.pi)) * (Cm - Cp)],
        [(-np.pi * g * Sp) / (L * g_p), Cp - delta_Kx, 0, (Sp / g_p)],
        [(-Sp / g_p), (L / (g * np.pi)) * (Cp - Cm), Cm - delta_Ky, (L / (g * np.pi)) * ((Sp / g_p) - (g_m * Sm))],
        [0, -(Sm / g_m), (np.pi * g * Sm) / (L * g_m), Cm]
    ])
    return M_core

# محاسبه ماتریس‌های چرخش R_positive و R_negative
def compute_R_positive(k, s):
    return np.array([
        [np.cos(k * s / 2), 0, -np.sin(k * s / 2), 0],
        [0, np.cos(k * s / 2), 0, -np.sin(k * s / 2)],
        [np.sin(k * s / 2), 0, np.cos(k * s / 2), 0],
        [0, np.sin(k * s / 2), 0, np.cos(k * s / 2)]
    ])

def compute_R_negative(k, s):
    return np.array([
        [np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N), 0],
        [0, np.cos(k * s / 2 * N), 0, np.sin(k * s / 2 * N)],
        [-np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N), 0],
        [0, -np.sin(k * s / 2 * N), 0, np.cos(k * s / 2 * N)]
    ])

# محاسبه ماتریس انتقال M_HQ
def helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m, delta_Kx, delta_Ky):
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    M_core = compute_M_core(L, g, g_p, g_m, s_out - s_in, delta_Kx, delta_Ky)
    M_HQ = R_out @ M_core @ R_in 
    return M_HQ

# تابع محاسبه‌ی ماتریس سیگما داخلی
def sigma_inner_matrix(k, K, A_H, A_L):
    sigma = np.zeros((4, 4))
    sigma[0, 0] = ((4 * k**2 + 4 * K) /(k**2)) * A_H**2 + (k**2 / K**2) * A_L**2
    sigma[1, 1] = 16 * (K**2 / k**2) * A_H**2
    sigma[2, 2] = 4 * A_H**2 + ((k**2 - 4 * K) / K**2) * A_L**2
    sigma[3, 3] = 4 * A_L**2
    sigma[0, 3] = sigma[3, 0] = 2 * (k / K) * A_L**2
    sigma[1, 2] = sigma[2, 1] = 8 * (K / k) * A_H**2
    return sigma

# اعمال ماتریس سیگما و محاسبه نهایی با ضرب ماتریس‌های چرخش
def sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out):
    sigma_inner = sigma_inner_matrix(k, K, A_H, A_L)
    
    # محاسبه ماتریس‌های چرخش
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    
    # اعمال چرخش‌ها به سیگما
    sigma_rotated = R_out @ sigma_inner @ R_in
    
    # محاسبه سیگمای نهایی با ضرب در M_HQ
    sigma_final = M_HQ @ sigma_rotated @ M_HQ.T
    return sigma_final

# محاسبه انتشار
def compute_emittance(cov_matrix):
    return np.sqrt(np.linalg.det(cov_matrix))

# محاسبه پارامترهای توییس
def compute_twist_parameters(cov_matrix, emittance):
    beta = cov_matrix[0, 0] / emittance
    alpha = -cov_matrix[0, 1] / emittance
    gamma = cov_matrix[1, 1] / emittance
    return beta, alpha, gamma

# محاسبه M_HQ
M_HQ = helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m, delta_Kx, delta_Ky)

# محاسبه ماتریس سیگمای اولیه
sigma_initial = sigma_inner_matrix(k, K, A_H, A_L)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های اولیه
cov_x_initial = sigma_initial[:2, :2]
cov_y_initial = sigma_initial[2:, 2:]

# محاسبه امیتنس‌های اولیه
emittance_x_initial = compute_emittance(cov_x_initial)
emittance_y_initial = compute_emittance(cov_y_initial)

# محاسبه پارامترهای توییس برای امیتنس‌های اولیه
beta_x_initial, alpha_x_initial, gamma_x_initial = compute_twist_parameters(cov_x_initial, emittance_x_initial)
beta_y_initial, alpha_y_initial, gamma_y_initial = compute_twist_parameters(cov_y_initial, emittance_y_initial)

# نمایش امیتنس‌های اولیه
print("\nInitial Emittance for x-plane:")
print(f"Emittance: {emittance_x_initial:.4f}")
print(f"Beta: {beta_x_initial:.4f}, Alpha: {alpha_x_initial:.4f}, Gamma: {gamma_x_initial:.4f}")

print("\nInitial Emittance for y-plane:")
print(f"Emittance: {emittance_y_initial:.4f}")
print(f"Beta: {beta_y_initial:.4f}, Alpha: {alpha_y_initial:.4f}, Gamma: {gamma_y_initial:.4f}")

# محاسبه سیگمای نهایی
sigma_final = sigma_matrix(M_HQ, k, K, A_H, A_L, s_in, s_out)

# جدا کردن ماتریس‌های کواریانس x و y برای امیتنس‌های نهایی
cov_x_final = sigma_final[:2, :2]
cov_y_final = sigma_final[2:, 2:]

# محاسبه امیتنس‌های نهایی
emittance_x_final = compute_emittance(cov_x_final)
emittance_y_final = compute_emittance(cov_y_final)

# محاسبه پارامترهای توییس برای امیتنس‌های نهایی
beta_x_final, alpha_x_final, gamma_x_final = compute_twist_parameters(cov_x_final, emittance_x_final)
beta_y_final, alpha_y_final, gamma_y_final = compute_twist_parameters(cov_y_final, emittance_y_final)

# نمایش نتایج نهایی برای x
print("\nFinal Twiss Parameters for x-plane (Transfer Matrix):")
print(f"Emittance: {emittance_x_final:.4f}")
print(f"Beta: {beta_x_final:.4f}, Alpha: {alpha_x_final:.4f}, Gamma: {gamma_x_final:.4f}")

# نمایش نتایج نهایی برای y
print("\nFinal Twiss Parameters for y-plane (Transfer Matrix):")
print(f"Emittance: {emittance_y_final:.4f}")
print(f"Beta: {beta_y_final:.4f}, Alpha: {alpha_y_final:.4f}, Gamma: {gamma_y_final:.4f}")



Initial Emittance for x-plane:
Emittance: 0.0007
Beta: 0.4355, Alpha: -0.0000, Gamma: 2.2963

Initial Emittance for y-plane:
Emittance: 0.0002
Beta: 1.1014, Alpha: -0.0000, Gamma: 0.9079

Final Twiss Parameters for x-plane (Transfer Matrix):
Emittance: 0.0098
Beta: 0.0436, Alpha: 2.8975, Gamma: 209.2309

Final Twiss Parameters for y-plane (Transfer Matrix):
Emittance: 0.0082
Beta: 40.0720, Alpha: -0.2442, Gamma: 0.0262
